In [ ]:
import zipfile
with zipfile.ZipFile('/content/gdrive/MyDrive/FYP/multi_classification_data.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/gdrive/MyDrive/FYP/multi_classification_data')

##### Import required Libraries

In [ ]:
import os 
import tensorflow as tf 
from google.colab import drive
import matplotlib.pyplot as plt
from tensorflow.keras import layers 
from tensorflow.keras.applications.vgg16 import VGG16 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
drive.mount('/content/gdrive')

##### Check if GPU is detected

In [ ]:
device_name = tf.test.gpu_device_name()
print(device_name)
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

##### Split the dataset into train and validation

In [ ]:
base_dir = '/content/gdrive/MyDrive/FYP/multi_classification_data/multi_classification_data'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

train_Intraventricular_dir = os.path.join(train_dir, 'Intraventricular')
validation_Intraventricular_dir = os.path.join(validation_dir, 'Intraventricular')

train_Normal_dir = os.path.join(train_dir, 'Normal')
validation_Normal_dir = os.path.join(validation_dir, 'Normal')

train_Subarachnoid_dir = os.path.join(train_dir, 'Subarachnoid')
validation_Subarachnoid_dir = os.path.join(validation_dir, 'Subarachnoid')

train_Chronic_dir = os.path.join(train_dir, 'Chronic')
validation_Chronic_dir = os.path.join(validation_dir, 'Chronic')

train_Intraparenchymal_dir = os.path.join(train_dir, 'Intraparenchymal')
validation_Intraparenchymal_dir = os.path.join(validation_dir, 'Intraparenchymal')

train_Epidural_dir = os.path.join(train_dir, 'Epidural')
validation_Epidural_dir = os.path.join(validation_dir, 'Epidural')

train_Subdural_dir = os.path.join(train_dir, 'c')
validation_Subdural_dir = os.path.join(validation_dir, 'Subdural')


##### Augment and generate images

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
valid_datagen = ImageDataGenerator( rescale = 1.0/255. )

In [ ]:
train_generator = train_datagen.flow_from_directory(train_dir, batch_size = 20, class_mode = 'categorical', target_size = (512, 512))
validation_generator = valid_datagen.flow_from_directory(validation_dir,  batch_size = 20, class_mode = 'categorical', target_size = (512, 512))

##### Defining base model and unfreezing layers

In [ ]:
base_model = VGG16(input_shape = (512, 512, 3),
                   include_top = False,
                   weights = 'imagenet')

trainable_blocks = ['block5_conv1', 'block5_conv2', 'block5_conv3']

for layer in base_model.layers:
    if layer.name not in trainable_blocks:
        layer.trainable = False
    else:
        layer.trainable = True

##### Adding projection layers to the backbone network

In [ ]:
x = layers.Flatten()(base_model.output)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(7, activation = None)(x) # 7 classes (Normal, Subdural, Subarachnoid, Intraparenchymal, Chronic (Hematoma), Intraventricular, Epidural)

##### Compiling the model and defining the optimizer, loss function, metrics

In [ ]:
with tf.device('/device:GPU:0'):
  model = tf.keras.models.Model(base_model.input, x)
  model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001),
                loss = tf.keras.losses.CategoricalCrossentropy(),
                metrics=[tf.keras.metrics.CategoricalAccuracy(), 'acc']
                )
  vgghist = model.fit(train_generator, validation_data = validation_generator, steps_per_epoch = 100, epochs = 10)

##### Plotting train and validation results

In [ ]:
acc = vgghist.history['acc']
val_acc = vgghist.history['val_categorical_accuracy']
loss = vgghist.history['loss']
val_loss = vgghist.history['val_loss']
epochs_range = range(10)

plt.figure(figsize=(15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

##### Evaluating the model on the test dataset

In [ ]:
test_dir = os.path.join(base_dir, 'test')
test_normal_dir = os.path.join(test_dir, 'Intraventricular')
test_abnormal_dir = os.path.join(test_dir, 'Normal')
test_normal_dir = os.path.join(test_dir, 'Subarachnoid')
test_abnormal_dir = os.path.join(test_dir, 'Chronic')
test_normal_dir = os.path.join(test_dir, 'Intraparenchymal')
test_abnormal_dir = os.path.join(test_dir, 'Epidural')
test_normal_dir = os.path.join(test_dir, 'Subdural')

test_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_generator = test_datagen.flow_from_directory(test_dir,  batch_size = 20, class_mode = 'categorical', target_size = (512, 512))
model.evaluate(test_generator)